In [2]:
#Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

data_path = "Resources/COVID_final_datas.csv"


data = pd.read_csv(data_path)

data.head()

,Unnamed: 0,Country,Country Code,Cases,Deaths,Deaths per 1000 Cases,Population 2019,"Cases per 100,000 People","Deaths per 100,000 People",2017 GDP in Billions,2018 GDP in Billions,2019 GDP in Billions,Capital,Latitude,Longitude
0,0,Afghanistan,AFG,39616,1470,37.11,38041757.0,104.14,3.86,20.191760,19.484385,19.101354,Kabul,34.5167,69.1833
1,1,Albania,ALB,14730,407,27.63,2862427.0,514.60,14.22,13.019693,15.147021,15.278077,Tirana,41.3275,19.8189
2,2,Algeria,DZA,52520,1779,33.87,43053054.0,121.99,4.13,167.000000,174.000000,170.000000,Algiers,36.7631,3.0506
3,3,Andorra,AND,2568,53,20.64,76177.0,3371.10,69.57,3.000181,3.218316,3.154058,Andorra la Vella,42.5,1.5165
4,4,Angola,AGO,5725,211,36.86,31825299.0,17.99,0.66,122.000000,101.000000,94.635416,Luanda,-8.8383,13.2344


In [ ]:
subset = data[['Country', 'Cases', 'Deaths', 'Deaths per 1000 Cases', 'Population 2019', 'Latitude']]
subset.head()

In [ ]:
bins = [0, 109999, 799999, 2399999, 4949999, 8499999, 11499999, 18999999, 34999999, 79999999, 1500000000]
pop_ranges = ['up to 110k', '110k-799k','800k-2.39m', '2.4m-4.94m', '4.95m-8.49m', '8.5m-11.49m', '11.5m-18.9m', '20m-34.9m', '35m-79.9m', '70m-1.5b']
subset['Population Range'] = pd.cut(subset['Population 2019'], bins, labels=pop_ranges, include_lowest=True)
subset.head()

In [ ]:
subset = subset.loc[subset['Latitude'] != 'Not Available']
subset['Latitude'].astype(float)


In [ ]:
#pop_range = '35m-79.9m'

group = subset.loc[(subset['Population Range'] == f'{pop_range}')]
group_lat = group['Latitude'].sort_values()
group_lat_abs = group['Latitude'].apply(lambda x: float(x)).abs()

group_cases = group['Cases']
group_deaths = group['Deaths']
group_dp1k = group['Deaths per 1000 Cases']

In [ ]:
case_cc_list = []
case_rs_list = []

deaths_cc_list = []
deaths_rs_list = []

mortality_cc_list = []
mortality_rs_list = []


for pop_range in pop_ranges:


    group = subset.loc[(subset['Population Range'] == f'{pop_range}')]
    group_lat = group['Latitude'].sort_values()
    group_lat_abs = group['Latitude'].apply(lambda x: float(x)).abs()

    group_cases = group['Cases']
    group_deaths = group['Deaths']
    group_dp1k = group['Deaths per 1000 Cases']


    plt.scatter(group_lat_abs, group_cases)
    plt.xlabel('Latitude Absolute Value')
    plt.ylabel('Total Cases')
    plt.title(f'Cases by Latitude (absolute value) for {pop_range}')


    slope, intercept, r, p, std_err = st.linregress(group_lat_abs, group_cases)
    fit = slope * group_lat_abs + intercept
    plt.plot(group_lat_abs,fit,color='r')

    plt.tight_layout()
    plt.savefig(f'Images/Lat vs Cases for{pop_range}.png')
    plt.show()
    
    rs = round(r**2,2)
    cc = round(st.pearsonr(group_lat_abs,group_cases)[0],2)
    case_cc_list.append(cc)
    case_rs_list.append(rs)
    
    print(f"r-squared : {rs}")
    print(f"Correlation Coefficient : {cc}")
    print('-------------------------------------------------------')
    
    #-------------------------------------------------------------------------------------------------------------------------
  
    plt.scatter(group_lat_abs, group_deaths)
    plt.xlabel('Latitude Absolute Value')
    plt.ylabel('Total Deaths')
    plt.title(f'Deaths by Latitude (absolute value) for {pop_range}')


    slope, intercept, r, p, std_err = st.linregress(group_lat_abs, group_deaths)
    fit = slope * group_lat_abs + intercept
    plt.plot(group_lat_abs,fit,color='r')


    plt.tight_layout()
    plt.savefig(f'Images/Lat vs Deaths for{pop_range}.png')
    plt.show()

    rs = round(r**2,2)
    cc = round(st.pearsonr(group_lat_abs,group_cases)[0],2)
    deaths_cc_list.append(cc)
    deaths_rs_list.append(rs)
    
    print(f"r-squared : {rs}")
    print(f"Correlation Coefficient : {cc}")
    print('-------------------------------------------------------')
    #-------------------------------------------------------------------------------------------------------------------------
    plt.subplots()
    plt.scatter(group_lat_abs, group_dp1k)
    plt.xlabel('Latitude Absolute Value')
    plt.ylabel('Deaths (per 1k Cases)')
    plt.title(f'Deaths (per 1k Cases) by Latitude (absolute value) for {pop_range}')


    slope, intercept, r, p, std_err = st.linregress(group_lat_abs, group_dp1k)
    fit = slope * group_lat_abs + intercept
    plt.plot(group_lat_abs,fit,color='r')


    plt.tight_layout()
    plt.savefig(f'Images/Lat vs Mortality for{pop_range}.png')
    plt.show()

    rs = round(r**2,2)
    cc = round(st.pearsonr(group_lat_abs,group_cases)[0],2)
    mortality_cc_list.append(cc)
    mortality_rs_list.append(rs)
    
    print(f"r-squared : {rs}")
    print(f"Correlation Coefficient : {cc}")



In [ ]:
cc_df = pd.DataFrame(list(zip(pop_ranges, case_cc_list, deaths_cc_list, mortality_cc_list)),
                       columns = ['Population', 'Cases', 'Deaths','Mortality'])
cc_df.set_index('Population')

In [ ]:
cc_df.plot(kind='bar')
plt.xlabel('Population Range')
plt.ylabel('Correlation Coefficient')
plt.title('Correlation Coefficient by Population Range')
ticks = range(0, 10)
plt.xticks(ticks, labels=pop_ranges, rotation=45)

plt.savefig('Images/correlation coefficient by pop.png')
plt.show()

In [ ]:
rs_df = pd.DataFrame(list(zip(pop_ranges, case_rs_list, deaths_rs_list, mortality_rs_list)),
                       columns = ['Population', 'Cases', 'Deaths','Mortality'])
rs_df.set_index('Population')

In [ ]:
rs_df.plot(kind='bar')
plt.xlabel('Population Range')
plt.ylabel('R-Squared Value')
plt.title('R-Squared Value by Population Range')
ticks = range(0, 10)
plt.xticks(ticks, labels=pop_ranges, rotation=45)

plt.savefig('Images/rs value by pop.png')
plt.show()

In [ ]:
pop_group = subset.groupby('Population Range')
avg_deaths = pop_group['Deaths per 1000 Cases'].mean()

In [ ]:
avg_deaths.plot(kind='bar')
plt.xlabel('Population Range')
plt.ylabel('Mortality Rate (deaths per 1000 cases)')
plt.title('Mortality Rate by Population Range')
plt.xticks(rotation=45)
plt.savefig('Images/mortality by pop.png')
plt.show()